In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings('ignore')

# Custom colormap for visualization
colors = [(1, 0, 0), (0, 0, 1)]  # Red to Blue
cmap_name = "red_blue"
red_blue_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)

# Data preparation and clustering functions (load_data, impute_sofa_day3, prepare_data, perform_kmeans, etc.)
# Data preparation functions
def load_data(filename):
    df = pd.read_csv(filename)
    return df

def impute_sofa_day3(row):
    if pd.isna(row['SOFA_D3']):
        if row['mortality'] == 1 and (row['dud'] == 2 or row['dud'] == 3):
            return 24
        else:
            return np.nan
    return row['SOFA_D3']

def prepare_data(df, columns_of_interest):
    df['SOFA_D3'] = df.apply(impute_sofa_day3, axis=1)
    df.dropna(subset=['SOFA_D3'], inplace=True)
    df[columns_of_interest] = np.log1p(df[columns_of_interest])
    return df

# Clustering Functions
def perform_kmeans(X_scaled, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    return labels

def all_subsets(columns):
    """Generate all non-empty combinations of the column list."""
    return chain(*[combinations(columns, i + 1) for i in range(len(columns))])

def perform_agglomerative(X_scaled, num_clusters):
    agglomerative = AgglomerativeClustering(n_clusters=num_clusters)
    labels = agglomerative.fit_predict(X_scaled)
    return labels

def perform_spectral(X_scaled, num_clusters):
    spectral = SpectralClustering(n_clusters=num_clusters, random_state=42)
    labels = spectral.fit_predict(X_scaled)
    return labels

def evaluate_clustering_with_labels(X_scaled, labels):
    # Check for noise labels produced by DBSCAN and handle them
    if -1 in labels:
        # Filter out noise points for metric calculation
        core_samples_mask = labels != -1
        X_scaled_core = X_scaled[core_samples_mask]
        labels_core = labels[core_samples_mask]
        if len(np.unique(labels_core)) <= 1:
            return -1, float('inf')  # Invalid scenario
    else:
        X_scaled_core = X_scaled
        labels_core = labels
        
    silhouette = silhouette_score(X_scaled_core, labels_core) if len(set(labels_core)) > 1 else -1
    davies_bouldin = davies_bouldin_score(X_scaled_core, labels_core) if len(set(labels_core)) > 1 else float('inf')
    return silhouette, davies_bouldin

# Function to iterate over all combinations of columns and evaluate clustering
def cluster_and_evaluate_all_combinations(df_subset, columns_of_interest, methods):
    best_overall_metrics = {'Silhouette': -1, 'Davies_Bouldin': float('inf'), 'Composite': -float('inf'),
                            'Columns': None, 'Method': None, 'Num_Clusters': None}

    imputer = SimpleImputer(strategy="mean")
    scaler = StandardScaler()

    for num_features in range(1, len(columns_of_interest) + 1):
        for cols in combinations(columns_of_interest, num_features):
            df_imputed = imputer.fit_transform(df_subset[list(cols)])
            X_scaled = scaler.fit_transform(df_imputed)

            for method in methods:
                for num_clusters in range(2, 11):
                    labels = None
                    if method == 'K-means':
                        labels = perform_kmeans(X_scaled, num_clusters)
                    elif method == 'Agglomerative':
                        labels = perform_agglomerative(X_scaled, num_clusters)
                    elif method == 'Spectral':
                        labels = perform_spectral(X_scaled, num_clusters)

                    silhouette, davies_bouldin = evaluate_clustering_with_labels(X_scaled, labels)
                    composite = silhouette - davies_bouldin

                    if composite > best_overall_metrics['Composite']:
                        best_overall_metrics.update({
                            'Silhouette': silhouette,
                            'Davies_Bouldin': davies_bouldin,
                            'Composite': composite,
                            'Columns': cols,
                            'Method': method,
                            'Num_Clusters': num_clusters
                        })

    return best_overall_metrics

def cluster_and_evaluate_all_combinations(df_subset, columns_of_interest, methods, imputer, scaler):
    best_overall_metrics = {'Silhouette': -1, 'Davies_Bouldin': float('inf'), 'Composite': -float('inf'),
                            'Columns': None, 'Method': None, 'Num_Clusters': None}

    for num_features in range(1, len(columns_of_interest) + 1):
        for cols in combinations(columns_of_interest, num_features):
            df_imputed = imputer.fit_transform(df_subset[list(cols)])
            X_scaled = scaler.fit_transform(df_imputed)
                      
            for method in methods:
                for num_clusters in range(2, 11):
                    labels = None
                    if method == 'K-means':
                        labels = perform_kmeans(X_scaled, num_clusters)
                    elif method == 'Agglomerative':
                        labels = perform_agglomerative(X_scaled, num_clusters)
                    elif method == 'Spectral':
                        labels = perform_spectral(X_scaled, num_clusters)

                    silhouette, davies_bouldin = evaluate_clustering_with_labels(X_scaled, labels)
                    composite = silhouette - davies_bouldin

                    if composite > best_overall_metrics['Composite']:
                        best_overall_metrics.update({
                            'Silhouette': silhouette,
                            'Davies_Bouldin': davies_bouldin,
                            'Composite': composite,
                            'Columns': cols,
                            'Method': method,
                            'Num_Clusters': num_clusters
                        })

    return best_overall_metrics
   
def cluster_and_evaluate(df_subset, columns_of_interest, methods):
    best_metrics_by_method = {}
    best_labels_by_method = {}

    imputer = SimpleImputer(strategy="mean")
    scaler = StandardScaler()
    df_imputed = imputer.fit_transform(df_subset[columns_of_interest])
    X_scaled = scaler.fit_transform(df_imputed)
    
    for method in methods:
        best_silhouette = -1
        best_davies_bouldin = float('inf')
        best_labels = None
        best_num_clusters = 0

        for num_clusters in range(2, 11):
            if method == 'K-means':
                labels = perform_kmeans(X_scaled, num_clusters)
            elif method == 'Agglomerative':
                labels = perform_agglomerative(X_scaled, num_clusters)
            elif method == 'Spectral':
                labels = perform_spectral(X_scaled, num_clusters)            

            silhouette, davies_bouldin = evaluate_clustering_with_labels(X_scaled, labels)
            if silhouette > best_silhouette:
                best_silhouette = silhouette
                best_labels = labels
                best_num_clusters = num_clusters

        composite = best_silhouette - davies_bouldin
        if best_labels is not None:
            best_metrics_by_method[method] = {'Silhouette': best_silhouette, 'Davies_Bouldin': davies_bouldin, 'Composite': composite, 'Num_Clusters': best_num_clusters}
            best_labels_by_method[method] = best_labels

    return best_metrics_by_method, best_labels_by_method

# Function for plotting scatter plot
def plot_and_save_scatter(df, method, column, labels, title, filename):
    plt.figure(figsize=(10, 6))
    unique_labels = np.unique(labels)
    palette = sns.color_palette("pastel", n_colors=len(unique_labels))  # More conservative color palette
    sns.scatterplot(x=df[column], y=df['SOFA_D3'], hue=labels, palette=palette)
    
    mean_values = df.groupby(labels)['SOFA_D3'].mean().to_dict()
    legend_labels = [f"Cluster {c}: mean SOFA_48h = {mean_values[c]:.2f}" for c in unique_labels]
    # Create custom handles for the legend
    handles = [mpatches.Patch(color=palette[i], label=legend_labels[i]) for i in range(len(unique_labels))]
    plt.legend(title='Cluster ID', handles=handles)

    plt.xlabel(column)
    plt.ylabel('SOFA_48h')
    plt.title(title)
    plt.savefig(filename, dpi=300)
    plt.show()

def plot_sofa_distribution_by_cluster_and_subset(df, labels, filename):
    df['Cluster'] = labels
    df['Sepsis'] = df['Sepsis'].replace({0: 'Non-Sepsis Patients', 1: 'Sepsis Patients'})

    g = sns.FacetGrid(df, col="Sepsis", row="Cluster", margin_titles=True, height=3)
    g.map(sns.kdeplot, "SOFA_D3", fill=True)

    g.set_titles(col_template="{col_name}", row_template="Cluster {row_name}")
    g.set_axis_labels("SOFA_48h", "Density")
    for ax in g.axes.flat:
        ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    # Removing the super title to avoid overlaying
    g.savefig(filename, dpi=300)
    plt.show()

if __name__ == "__main__":
    df = load_data('20240104_Data_combined_python_D1.csv')
    df = prepare_data(df, columns_of_interest)
    methods = ['K-means', 'Agglomerative', 'Spectral']

    imputer = SimpleImputer(strategy="mean")
    scaler = StandardScaler()

    for subset_name, df_subset in [("Septic", df[df['Sepsis'] == 1]), ("Non-Septic", df[df['Sepsis'] == 0]), ("All Patients", df)]:
        best_metrics = cluster_and_evaluate_all_combinations(df_subset, columns_of_interest, methods, imputer, scaler)
        print(f"Best Metrics for {subset_name}: {best_metrics}")
        
        method = best_metrics['Method']
        num_clusters = best_metrics['Num_Clusters']
        best_cols = best_metrics['Columns']
        df_imputed = imputer.fit_transform(df_subset[list(best_cols)])
        X_scaled = scaler.fit_transform(df_imputed)

        labels = None
        if method == 'K-means':
            labels = perform_kmeans(X_scaled, num_clusters)
        elif method == 'Agglomerative':
            labels = perform_agglomerative(X_scaled, num_clusters)
        elif method == 'Spectral':
            labels = perform_spectral(X_scaled, num_clusters)

        plot_column = best_cols[0]
        plot_and_save_scatter(df_subset, method, plot_column, labels, f"{subset_name} Scatter Plot - {plot_column} vs SOFA_48h", f"{subset_name}_scatter_plot_{plot_column}.png")
        plot_sofa_distribution_by_cluster_and_subset(df_subset, labels, f"{subset_name}_SOFA_D3_distribution.png")



if __name__ == "__main__":
    df = load_data('20240104_Data_combined_python_D1.csv')
    df = prepare_data(df, columns_of_interest)
    methods = ['K-means', 'Agglomerative', 'Spectral']

    imputer = SimpleImputer(strategy="mean")
    scaler = StandardScaler()

    for subset_name, df_subset in [("Septic", df[df['Sepsis'] == 1]), ("Non-Septic", df[df['Sepsis'] == 0]), ("All Patients", df)]:
        print(f"----- {subset_name} -----")
        for method in methods:
            best_metrics_by_method, best_labels_by_method = cluster_and_evaluate(df_subset, columns_of_interest, [method])
            
            best_metrics = best_metrics_by_method[method]
            print(f"Method: {method}")
            print(f"Silhouette: {best_metrics['Silhouette']}")
            print(f"Davies-Bouldin: {best_metrics['Davies_Bouldin']}")
            print(f"Best Columns: {best_metrics['Columns']}")
            print(f"Number of Clusters: {best_metrics['Num_Clusters']}\n")

            method = best_metrics['Method']
            num_clusters = best_metrics['Num_Clusters']
            best_cols = best_metrics['Columns']
            df_imputed = imputer.fit_transform(df_subset[list(best_cols)])
            X_scaled = scaler.fit_transform(df_imputed)

            labels = None
            if method == 'K-means':
                labels = perform_kmeans(X_scaled, num_clusters)
            elif method == 'Agglomerative':
                labels = perform_agglomerative(X_scaled, num_clusters)
            elif method == 'Spectral':
                labels = perform_spectral(X_scaled, num_clusters)

            plot_column = best_cols[0]
            plot_and_save_scatter(df_subset, method, plot_column, labels, f"{subset_name} Scatter Plot - {plot_column} vs SOFA_48h", f"{subset_name}_scatter_plot_{plot_column}.png")
            plot_sofa_distribution_by_cluster_and_subset(df_subset, labels, f"{subset_name}_SOFA_D3_distribution.png")